This notebook will pull from the FPL API the weekly results for each player and store it in a dictionary

In [11]:
import pandas as pd
import numpy as np
import requests

In [5]:
top500 = pd.read_csv('top500.csv')
top500.head()

,id,assists,bonus,bps,chance_of_playing_next_round,chance_of_playing_this_round,clean_sheets,code,cost_change_event,cost_change_event_fall,...,threat,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,yellow_cards
0,215,9,10,246,75.0,0.0,3,61366,0,0,...,168.0,63,2191000,98273,1321505,155038,0.7,6.3,De Bruyne,0
1,210,3,6,255,NaN,NaN,2,37572,0,0,...,392.0,61,1186805,45823,518527,49181,0.4,5.0,Agüero,0
2,192,2,8,162,100.0,100.0,3,110979,1,-1,...,260.0,57,1401900,362810,637616,12568,0.7,4.9,Mané,0
3,191,3,7,150,75.0,NaN,3,118748,0,0,...,464.0,55,838413,10349,1276908,363221,0.4,4.4,Salah,0
4,11,1,10,215,NaN,NaN,2,54694,1,-1,...,384.0,55,950071,138152,856519,28407,0.7,5.0,Aubameyang,2


In [12]:
histories = {}

ids = top500['id']

for id_ in ids:
    url = 'https://fantasy.premierleague.com/api/element-summary/' + str(id_) + '/'
    r = requests.get(url)
    player_data = r.json()
    player_history = player_data['history']
    histories[id_] = player_history

In [29]:
history_dfs = {}

for key in histories:
    history_dfs[key] = pd.DataFrame(histories[key])                                

In [33]:
type(history_dfs)

dict

In [40]:
top500histories = pd.concat(history_dfs.values(), ignore_index=True)
top500histories.head(10)

,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,ict_index,...,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards
0,1,1,37,1,37.4,215,8,0,0,10.2,...,5,0,27.0,7,0,0,0,95,False,0
1,2,3,44,0,127.7,215,16,2,0,22.5,...,2,2,38.0,11,2049,81457,79408,95,True,0
2,1,0,22,0,42.3,215,22,1,0,7.6,...,3,1,16.0,5,460538,510105,49567,96,False,0
3,1,2,41,1,26.7,215,37,0,1,11.4,...,0,4,27.0,13,293070,373295,80225,97,True,0
4,0,0,6,0,36.8,215,46,0,0,4.8,...,2,3,2.0,1,399792,438722,38930,98,False,0
5,2,3,66,1,109.4,215,57,0,1,23.8,...,0,8,40.0,17,35783,135252,99469,98,True,0
6,2,1,30,0,33.6,215,65,1,0,7.5,...,3,1,18.0,9,388729,412503,23774,99,False,0
7,0,0,0,0,0.0,215,75,0,0,0.0,...,2,0,0.0,0,-653701,141393,795094,100,True,0
8,0,0,27,0,0.3,210,8,0,1,5.1,...,5,0,17.0,5,0,0,0,120,False,0
9,0,0,27,0,10.9,210,16,2,1,9.3,...,2,2,51.0,6,-103549,19746,123295,120,True,0


In [39]:
top500histories.to_csv('top500histories.csv')